In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000')

In [2]:
# !pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple
import pandas as pd
data = pd.read_csv(data_file)
print(data, type(data))

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000 <class 'pandas.core.frame.DataFrame'>


In [3]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]

# 填充数值列的缺失值
inputs['NumRooms'] = inputs['NumRooms'].replace('NA', pd.NA).astype(float)
inputs['NumRooms'] = inputs['NumRooms'].fillna(inputs['NumRooms'].mean())


# 查看结果
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


In [9]:
inputs.to_numpy() # 去掉行名称和列名称

array([[3.0, True, False],
       [2.0, False, True],
       [4.0, False, True],
       [3.0, False, True]], dtype=object)

In [5]:
# 转换成张量格式
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

In [6]:
# 计算每列的缺失值数量
missing_values_count = data.isna().sum()

# 找到缺失值最多的列
max_missing_col = missing_values_count.idxmax()
max_missing_col

'Alley'

In [7]:
data = data.drop(columns=[max_missing_col])
data

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000
